# AI Image Recognition of Sign Language and Sentiment Analysis using NLP

For persons who have hearing loss and trouble communicating, **Sign language** is vital. We employ this project to figure out what they're trying to convey. In this work we will classify images of static letter hand posture using **Neural Networks** and thereby using a **Text Classification Model** to analyse the emotions of handicapped people.

#### Importing the required libraries

In [1]:
import numpy as np # linear algebra
import os

import glob

from pathlib import Path
import pandas as pd

from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import pickle
from PIL import Image

!pip install --upgrade pip
!pip install transformers --user
from transformers import BertTokenizer, TFBertModel
!pip install --upgrade tensorflow

#### Testing Sign Language Model

In [2]:
# Creating a series of alphabets from a-z
letters = np.array(['a', 'b', 'c','d', 'e', 'f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
counts = pd.Series(letters)

In [3]:
# Loading the saved trained Sign Language Predictor Model
sign_model = load_model('SignLanguagePredictor.h5')

In [28]:
# Reading the images in a sequence
resized_images = []
for filename in sorted(glob.glob(os. getcwd() + "/project_test/worry_test/*.jpg")):
    img = Image.open(filename)
    # resizing the shape of image
    image = img.resize((224,224))
    resized_images.append(image)

In [29]:
res = ""
for i in resized_images:
    img_batch = np.expand_dims(i, axis=0)
    img_preprocessed = preprocess_input(img_batch)
    # predicting the letter using the model
    prediction = sign_model.predict(img_preprocessed)
    # concatenating each letter to a empty String
    res = res + counts[np.argmax(prediction)]
print(res)

1/1 [==============================] - 0s 35ms/step
worry


#### Testing Model for Emotion Detection

In [6]:
# Loading the saved trained Emotion Predictor Model
emotion_model = load_model('EmotionPredictor.h5',custom_objects={'TFBertModel': TFBertModel})

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [30]:
# Tokenizing the result text
x_test = tokenizer(text = [res],
                  padding='max_length',
                    truncation=True,
                    max_length=130,
                    return_tensors="tf")

In [10]:
# Creating a dictionary of Emotions
dic = {'anger':0,'fear':1,'joy':2,'love':3,'sadness':4,'suprise':5}
dic1 = list(dic.keys())

In [31]:
# Predicting the emotion from the model
predicted_raw = emotion_model.predict({'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']})

1/1 [==============================] - 0s 222ms/step


In [32]:
y_predicted = np.argmax(predicted_raw,axis=1)
# Printing the emotion
print("The Emotion detected from the result is "+dic1[y_predicted[0]])

The Emotion detected from the result is fear
